### Get data from .mat format to dict 
and save as python-friendly format

nina 2020.3.4

In [1]:
import scipy.io
matfile_path = '../S2_data/data.mat'
mat = scipy.io.loadmat(matfile_path)

In [2]:
mdata = mat['data']

In [3]:
mdata.dtype.names
# mdata['AB']

('AB',
 'AG',
 'VB',
 'VG',
 'AVB',
 'AVG',
 'AVFus',
 'dim_order',
 'response_order',
 'nsubjects')

In [4]:
print(mdata['AB'][0,0].shape)
print(mdata['AB'][0,0]['counts'][0,0].shape)

(1, 1)
(16, 3, 3)


In [15]:
ndata = {n: mdata[n][0,0] for n in mdata.dtype.names}
dict_data = {}
for each in ndata:
    print('processing {}'.format(each))
    each_d = ndata[each]
    if len(each_d.dtype) == 0:
        dict_data[each] = each_d
        continue
    each_d_n = {n: each_d[n][0,0] for n in each_d.dtype.names}
    
    dict_tmp={}
    for each_2 in each_d_n:
        print('processing {}'.format(each_2))
        each_dd = each_d_n[each_2]
        if len(each_dd.dtype) == 0:
            dict_tmp[each_2] = each_dd
            continue
        
        each_d_nn = {n: each_dd[n][0,0] for n in each_dd.dtype.names}
        dict_tmp[each_2] = each_d_nn
    
    dict_data[each] = dict_tmp

    
print('pre-processing done')

processing AB
processing counts
processing props
processing snr
processing AG
processing counts
processing props
processing snr
processing VB
processing counts
processing props
processing snr
processing VG
processing counts
processing props
processing snr
processing AVB
processing synch
processing asynch
processing AVG
processing synch
processing asynch
processing AVFus
processing synch
processing asynch
processing dim_order
processing response_order
processing nsubjects
pre-processing done


In [16]:
dict_data.keys()

dict_keys(['AB', 'AG', 'VB', 'VG', 'AVB', 'AVG', 'AVFus', 'dim_order', 'response_order', 'nsubjects'])

In [20]:
dict_data['AVG'].keys()

dict_keys(['synch', 'asynch'])

In [21]:
dict_data['AVG']['synch'].keys()

dict_keys(['counts', 'props', 'snr'])

In [30]:
dict_data['AVFus']['asynch']['counts']

array([[[ 6,  0, 19],
        [ 7,  0, 18],
        [ 4,  0, 21],
        [ 4,  2, 19],
        [ 5,  3, 17]],

       [[14,  0, 11],
        [14,  1, 10],
        [13,  0, 12],
        [13,  1, 11],
        [19,  0,  6]],

       [[ 7,  4, 14],
        [ 7,  4, 14],
        [13,  1, 11],
        [ 5, 10, 10],
        [ 8, 14,  3]],

       [[15,  0, 10],
        [13,  0, 12],
        [ 8,  0, 17],
        [22,  0,  3],
        [24,  0,  1]],

       [[ 8, 16,  1],
        [10,  9,  6],
        [ 7,  7, 11],
        [ 3, 19,  3],
        [ 6, 19,  0]],

       [[ 0,  0, 25],
        [ 2,  0, 23],
        [ 0,  0, 25],
        [ 2,  0, 23],
        [ 7,  0, 18]],

       [[ 8,  1, 16],
        [ 6,  0, 19],
        [ 4,  0, 21],
        [ 9,  3, 13],
        [16,  2,  7]],

       [[ 1, 10, 14],
        [ 2,  4, 19],
        [ 3,  3, 19],
        [ 5,  7, 13],
        [15,  5,  5]],

       [[ 8, 15,  2],
        [ 3, 18,  4],
        [ 4, 16,  5],
        [ 1, 21,  3],
        [ 7, 18,

##### Store the data into python-friendly format

1. store the dict data into pickle format.

In [31]:
import pickle

pkl_path = '../S2_data/data.pkl'
with open(pkl_path, 'wb') as f:
    pickle.dump(dict_data, f, pickle.HIGHEST_PROTOCOL)


try to read this file again:

In [32]:
with open(pkl_path, 'rb') as f:
    data_newv = pickle.load(f)

In [33]:
data_newv.keys()

dict_keys(['AB', 'AG', 'VB', 'VG', 'AVB', 'AVG', 'AVFus', 'dim_order', 'response_order', 'nsubjects'])

2. store it into npy format

In [34]:
import numpy as np
npy_filepath  = '../S2_data/data.npy'
np.save(npy_filepath, dict_data) 

In [35]:
# fix the 'Object arrays cannot be loaded when allow_pickle=False' problem
# https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56062555
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# # call load_data with allow_pickle implicitly set to true
read_dict = np.load(npy_filepath).item()

# restore np.load for future normal usage
np.load = np_load_old

In [36]:
read_dict.keys()

dict_keys(['AB', 'AG', 'VB', 'VG', 'AVB', 'AVG', 'AVFus', 'dim_order', 'response_order', 'nsubjects'])